### Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install opencv-contrib-python==3.4.2.17
!pip install --upgrade imutils

     |████████████████████████████████| 30.6MB 103kB/s 
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30
Requirement already up-to-date: imutils in /usr/local/lib/python3.6/dist-packages (0.5.3)


In [ ]:
import cv2
import numpy as np
import imutils
import matplotlib.pyplot as plt
from random import randrange
from google.colab.patches import cv2_imshow
import sys
import time
import os
import tqdm
from moviepy.editor import ImageSequenceClip

# For 24->8bit
from PIL import Image

%matplotlib inline

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1384448/45929032 bytes (3.0%)4087808/45929032 bytes (8.9%)8142848/45929032 bytes (17.7%)12410880/45929032 bytes (27.0%)16015360/45929032 bytes (34.9%)20324352/45929032 bytes (44.3%)24633344/45929032 bytes (53.6%)28966912/45929032 bytes (63.1%)33349632/45929032 bytes (72.6%)37601280/45929032 bytes (81.9%)41926656/45929032 bytes (91.3%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root/.im

#### For Stitcher


In [ ]:
def cropping(img):
  # create a 10 pixel border surrounding the stitched image
  print("Cropping...")
  stitched = cv2.copyMakeBorder(stitched, 10, 10, 10, 10,
    cv2.BORDER_CONSTANT, (0, 0, 0))

  # convert the stitched image to grayscale and threshold it
  # such that all pixels greater than zero are set to 255
  # (foreground) while all others remain 0 (background)
  gray = cv2.cvtColor(stitched, cv2.COLOR_BGR2GRAY)
  thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)[1]

  # To get the intermediate results
  plt.imshow(gray, cmap='gray')
  plt.title("Gray Image")
  plt.show()

  plt.pause(0.2)

  plt.imshow(thresh, cmap='gray')
  plt.title("Threshold Image")
  plt.show()

  plt.pause(0.2)

  # find all external contours in the threshold image then find
  # the *largest* contour which will be the contour/outline of
  # the stitched image
  cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)
  c = max(cnts, key=cv2.contourArea)

  # allocate memory for the mask which will contain the
  # rectangular bounding box of the stitched image region
  mask = np.zeros(thresh.shape, dtype="uint8")
  (x, y, w, h) = cv2.boundingRect(c)

  h = int(0.95*h)

  cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1)

  plt.imshow(cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1), cmap='gray')
  plt.title("Rectangular Mask")
  plt.show()

  plt.pause(0.2)

  # create two copies of the mask: one to serve as our actual
  # minimum rectangular region and another to serve as a counter
  # for how many pixels need to be removed to form the minimum
  # rectangular region
  minRect = mask.copy()
  sub = mask.copy()

  # # keep looping until there are no non-zero pixels left in the
  # # subtracted image
  # while cv2.countNonZero(sub) > 2500:
  #   # erode the minimum rectangular mask and then subtract
  #   # the thresholded image from the minimum rectangular mask
  #   # so we can count if there are any non-zero pixels left
  #   minRect = cv2.erode(minRect, None)
  #   sub = cv2.subtract(minRect, thresh)

  plt.imshow(minRect, cmap='gray')
  plt.title('Minimized Image/Box')
  plt.show()

  plt.pause(0.2)

  # find contours in the minimum rectangular mask and then
  # extract the bounding box (x, y)-coordinates
  cnts = cv2.findContours(minRect.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)
  c = max(cnts, key=cv2.contourArea)
  (x, y, w, h) = cv2.boundingRect(c)

  print(x, y, w, h)
  # use the bounding box coordinates to extract the our final
  # stitched image
  stitched_cropped = stitched[100: 60 + 906, x:x + w]

  # Display the output stitched image to our screen
  return stitched_cropped

In [ ]:
def stitch_gt(img_arr, stitcher):
  (status, stitched) = stitcher.stitch(img_arr)

  if (status==cv2.STITCHER_OK):
    # print('Panorama Generated')
    stitched = cv2.cvtColor(stitched, cv2.COLOR_BGR2RGB)
    # plt.imshow(stitched)
    # plt.pause(0.2)
    return stitched

  else:
    # print('Error Generating Panorama')
    pass

In [ ]:
def run(left_video, mid_video, right_video, stitcher):
  
  # Get information about the videos
  n_frames = min( int(left_video.get(cv2.CAP_PROP_FRAME_COUNT)),
                  int(mid_video.get(cv2.CAP_PROP_FRAME_COUNT)),
                  int(right_video.get(cv2.CAP_PROP_FRAME_COUNT)) )
  
  fps = int(left_video.get(cv2.CAP_PROP_FPS))
  frames = []

  count = -1
  step_to_continue_from = 2499
  fail_count = 0

  for _ in tqdm.tqdm(np.arange(n_frames)):
    
    count += 1

    if (count>-1 and count<4000):
      # Grab the frames from their respective video streams
      ok, left = left_video.read()
      _, mid = mid_video.read()
      _, right = right_video.read()

      if (count>step_to_continue_from) and ok:
        left = cv2.cvtColor(left, cv2.COLOR_BGR2RGB)
        mid = cv2.cvtColor(mid, cv2.COLOR_BGR2RGB)
        right = cv2.cvtColor(right, cv2.COLOR_BGR2RGB)

        # Stitch the frames together to form the panorama
        stitched_frame = stitch_gt([left, mid, right], stitcher)
        
        # No homography could not be computed
        if stitched_frame is None:
          print("[INFO]: Homography could not be computed!")
          
          fail_count+=1

          if fail_count>100:
            print("Stopping the process!")
            break
          
          else:
            continue

        # Add frame to video
        # stitched_frame = imutils.resize(stitched_frame, width=800)
        stitched_frame = cv2.resize(stitched_frame, (1920, 720))
        frames.append(stitched_frame)

    elif (count==4000):
      print("Finished Processing!")

  cv2.destroyAllWindows()
  return frames

In [ ]:
# Defining the stitcher
stitcher = cv2.createStitcher(try_use_gpu=True)

In [ ]:
# Defining the paths

left_video_in_path='/content/drive/My Drive/Colab Notebooks/Carla Dataset/City/Left.mp4'
mid_video_in_path='/content/drive/My Drive/Colab Notebooks/Carla Dataset/City/Front.mp4'
right_video_in_path='/content/drive/My Drive/Colab Notebooks/Carla Dataset/City/Right.mp4'

In [ ]:
# Set up video capture
left_video = cv2.VideoCapture(left_video_in_path)
mid_video = cv2.VideoCapture(mid_video_in_path)
right_video = cv2.VideoCapture(right_video_in_path)
print('[INFO]: {}, {} and {} loaded'.format(  left_video_in_path.split('/')[-1],
                                              mid_video_in_path.split('/')[-1],
                                              right_video_in_path.split('/')[-1])  )

print('[INFO]: Video stitching starting....')

# The frames of the stitched images
frames = run(left_video, mid_video, right_video, stitcher)

  0%|          | 1/14850 [00:00<26:54,  9.20it/s]

[INFO]: Left.mp4, Front.mp4 and Right.mp4 loaded
[INFO]: Video stitching starting....


 17%|█▋        | 2508/14850 [01:22<1:03:12,  3.25it/s]

[INFO]: Homography could not be computed!


 17%|█▋        | 2511/14850 [01:23<1:04:51,  3.17it/s]

...

[INFO]: Homography could not be computed!


 27%|██▋       | 3955/14850 [15:25<2:15:04,  1.34it/s]

[INFO]: Homography could not be computed!


100%|██████████| 14850/14850 [15:53<00:00, 15.58it/s] 

Finished Processing!


In [ ]:
print(len(frames))

1418


In [ ]:
print('[INFO]: Video stitching finished')

'''Writing via CV2'''
video_out_path='/content/drive/Shared drives/drive 4/Carla Dataset/Ground Truth'

for i in range(len(frames)):
  cv2.imwrite(video_out_path+f"/{i+2462}"+".png", frames[i])
  print("Creating : ",video_out_path,f"/{i+2462}",".png")

[INFO]: Video stitching finished
Creating :  /content/drive/Shared drives/drive 4/Carla Dataset/Ground Truth /2462 .png
Creating :  /content/drive/Shared drives/drive 4/Carla Dataset/Ground Truth /2463 .png
Creating :  /content/drive/Shared drives/drive 4/Carla Dataset/Ground Truth /2464 .png
Creating :  /content/drive/Shared drives/drive 4/Carla Dataset/Ground Truth /2465 .png
Creating :  /content/drive/Shared drives/drive 4/Carla Dataset/Ground Truth /2466 .png
Creating :  /content/drive/Shared drives/drive 4/Carla Dataset/Ground Truth /2467 .png
Creating :  /content/drive/Shared drives/drive 4/Carla Dataset/Ground Truth /2468 .png
Creating :  /content/drive/Shared drives/drive 4/Carla Dataset/Ground Truth /2469 .png
Creating :  /content/drive/Shared drives/drive 4/Carla Dataset/Ground Truth /2470 .png
Creating :  /content/drive/Shared drives/drive 4/Carla Dataset/Ground Truth /2471 .png
Creating :  /content/drive/Shared drives/drive 4/Carla Dataset/Ground Truth /2472 .png
Creating :